In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import scale

In [25]:
class FashionMNISTDataset(Dataset):
    def __init__(self, csv_file, transform = None):
        data = pd.read_csv(csv_file)
        self.X = np.array(data.iloc[:, 1:]).reshape(-1, 1, 28, 28).astype(float)
        
        self.Y = np.array(data.iloc[:, 0])

        del data;
        self.transform = transform
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        item = self.X[idx]
        label = self.Y[idx]
        
        if self.transform:
            item = self.transform(item)
            
        return (item, label)

In [26]:
num_epochs = 10
learning_rate = 0.001
batch_size = 100

In [27]:
train_dataset = FashionMNISTDataset(csv_file = 
                                    '/media/user/D:/Image Recognition QSTP/fashion-mnist_train.csv')
test_dataset = FashionMNISTDataset(csv_file = 
                                    '/media/user/D:/Image Recognition QSTP/fashion-mnist_test.csv')

In [28]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, 
                                         batch_size = batch_size,
                                         shuffle = False)

In [29]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
                    nn.BatchNorm2d(1),
                    nn.Conv2d(1, 16, kernel_size = 5, padding = 2),
                    nn.BatchNorm2d(16),
                    nn.ReLU(),
                    nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
                    nn.Conv2d(16, 32, kernel_size = 5, padding = 2),
                    nn.BatchNorm2d(32),
                    nn.ReLU(),
                    nn.MaxPool2d(2))
        self.fc = nn.Linear(32*7*7, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [30]:
model = CNN()

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [31]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = Variable(images.float())
        labels = Variable(labels)
        
        optimizer.zero_grad()
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
         
        if (i+1) % 100 == 0:
           print('Epoch : %d/%d, Iter : %d/%d, Loss: %.4f'
                %(epoch + 1, num_epochs, i + 1, len(train_dataset) // batch_size, loss.data[0])) 

Epoch : 1/10, Iter : 100/600, Loss: 0.3938
Epoch : 1/10, Iter : 200/600, Loss: 0.3515
Epoch : 1/10, Iter : 300/600, Loss: 0.3404
Epoch : 1/10, Iter : 400/600, Loss: 0.4006
Epoch : 1/10, Iter : 500/600, Loss: 0.1676
Epoch : 1/10, Iter : 600/600, Loss: 0.3573
Epoch : 2/10, Iter : 100/600, Loss: 0.2557
Epoch : 2/10, Iter : 200/600, Loss: 0.5157
Epoch : 2/10, Iter : 300/600, Loss: 0.2654
Epoch : 2/10, Iter : 400/600, Loss: 0.3247
Epoch : 2/10, Iter : 500/600, Loss: 0.1414
Epoch : 2/10, Iter : 600/600, Loss: 0.3126
Epoch : 3/10, Iter : 100/600, Loss: 0.2755
Epoch : 3/10, Iter : 200/600, Loss: 0.3203
Epoch : 3/10, Iter : 300/600, Loss: 0.3189
Epoch : 3/10, Iter : 400/600, Loss: 0.1978
Epoch : 3/10, Iter : 500/600, Loss: 0.1513
Epoch : 3/10, Iter : 600/600, Loss: 0.1806
Epoch : 4/10, Iter : 100/600, Loss: 0.2969
Epoch : 4/10, Iter : 200/600, Loss: 0.2739
Epoch : 4/10, Iter : 300/600, Loss: 0.2143
Epoch : 4/10, Iter : 400/600, Loss: 0.1557
Epoch : 4/10, Iter : 500/600, Loss: 0.2402
Epoch : 4/1

In [32]:
model.eval()
correct = 0
total = 0

for images, labels in test_loader:
    
    images = Variable(images.float())
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (labels == predicted).sum()
    
print('Test Accuracy of the model: %.4f %%' %(100 * correct / total)) 

Test Accuracy of the model: 91.6900 %
